In [1]:
import requests

url = "https://www.serebii.net/brilliantdiamondshiningpearl/sinnohpokedex.shtml"
response = requests.get(url)
html = response.content

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

In [3]:
rows = soup.find_all('tr')

In [4]:
pokemon_rows = rows[3:]

In [5]:
import pandas as pd

pokemon_df = df = pd.DataFrame(columns=['Pokédex no.', 'Name', 'URL'])

In [6]:
print(pokemon_df)

Empty DataFrame
Columns: [Pokédex no., Name, URL]
Index: []


In [7]:
extract_types = []
for index, row in enumerate(pokemon_rows):
    if index % 2 == 0:
        cells = row.find_all('td')
        if cells:
            for item_index, item in enumerate(cells):
                if item_index == 0:
                    dex_num = item.text.strip()
                elif item_index == 3:
                    name = item.text.strip()
        url = row.find_all('a', href=True)[0]['href']
        name = url[14:len(url)-1]
        types = []
        hrefs = row.find_all('a', href=True)
        for href_index, href in enumerate(hrefs):
            if href_index > 1:
                types.append(href['href'])
        extract_types.append(types)
        new_row = pd.DataFrame([{'Pokédex no.':dex_num, 'Name':name, 'URL': url}])
        pokemon_df = pd.concat([pokemon_df, new_row])
        
print(pokemon_df)

   Pokédex no.      Name                      URL
0         #001   turtwig   /pokedex-swsh/turtwig/
0         #002    grotle    /pokedex-swsh/grotle/
0         #003  torterra  /pokedex-swsh/torterra/
0         #004  chimchar  /pokedex-swsh/chimchar/
0         #005  monferno  /pokedex-swsh/monferno/
..         ...       ...                      ...
0         #147   mesprit   /pokedex-swsh/mesprit/
0         #148     azelf     /pokedex-swsh/azelf/
0         #149    dialga    /pokedex-swsh/dialga/
0         #150    palkia    /pokedex-swsh/palkia/
0         #151   manaphy   /pokedex-swsh/manaphy/

[151 rows x 3 columns]


In [8]:
types_mapping = {'/pokedex-swsh/water.shtml': 'Water',
                 '/pokedex-swsh/fire.shtml': 'Fire',
                 '/pokedex-swsh/grass.shtml': 'Grass',
                 '/pokedex-swsh/flying.shtml': 'Flying',
                 '/pokedex-swsh/fighting.shtml': 'Fighting',
                 '/pokedex-swsh/poison.shtml': 'Poison',
                 '/pokedex-swsh/electric.shtml': 'Electric',
                 '/pokedex-swsh/ground.shtml': 'Ground',
                 '/pokedex-swsh/rock.shtml': 'Rock',
                 '/pokedex-swsh/psychic.shtml': 'Psychic',
                 '/pokedex-swsh/ice.shtml': 'Ice',
                 '/pokedex-swsh/bug.shtml': 'Bug',
                 '/pokedex-swsh/ghost.shtml': 'Ghost',
                 '/pokedex-swsh/steel.shtml': 'Steel',
                 '/pokedex-swsh/dragon.shtml': 'Dragon',
                 '/pokedex-swsh/dark.shtml': 'Dark',
                 '/pokedex-swsh/fairy.shtml': 'Fairy',
                 '/pokedex-swsh/normal.shtml': 'Normal'}
types = []
for type_list in extract_types:
    lst = []
    for item in type_list:
        lst.append(types_mapping.get(item))
    types.append(lst)
print(len(types))

151


In [9]:
pokemon_df['Type'] = types
print(pokemon_df)

   Pokédex no.      Name                      URL              Type
0         #001   turtwig   /pokedex-swsh/turtwig/           [Grass]
0         #002    grotle    /pokedex-swsh/grotle/           [Grass]
0         #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]
0         #004  chimchar  /pokedex-swsh/chimchar/            [Fire]
0         #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]
..         ...       ...                      ...               ...
0         #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]
0         #148     azelf     /pokedex-swsh/azelf/         [Psychic]
0         #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]
0         #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]
0         #151   manaphy   /pokedex-swsh/manaphy/           [Water]

[151 rows x 4 columns]


In [10]:
pokemon_df.reset_index(drop=True, inplace=True)
print(pokemon_df)

    Pokédex no.      Name                      URL              Type
0          #001   turtwig   /pokedex-swsh/turtwig/           [Grass]
1          #002    grotle    /pokedex-swsh/grotle/           [Grass]
2          #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]
3          #004  chimchar  /pokedex-swsh/chimchar/            [Fire]
4          #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]
..          ...       ...                      ...               ...
146        #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]
147        #148     azelf     /pokedex-swsh/azelf/         [Psychic]
148        #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]
149        #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]
150        #151   manaphy   /pokedex-swsh/manaphy/           [Water]

[151 rows x 4 columns]


In [11]:
keys = ['Base Stat Total', 'HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed']
cols = ['Name', 'Base Stat Total', 'HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed']
stats_df = pd.DataFrame(columns=keys)
types = []
for index, row in pokemon_df.iterrows():
    part = row['URL']
    url = "https://www.serebii.net" + part
    response = requests.get(url)
    html = response.content
    pokedex_soup = BeautifulSoup(html, 'html.parser')
    pokedex_rows = pokedex_soup.find_all('tr')
    for index, pokedex_row in enumerate(pokedex_rows):
        if 'Base Stats' in pokedex_row.text:
            stats = pokedex_row.text.strip().split('\n')
            newlst = [dict(zip(keys, values)) for values in [iter(stats)]]
            newlst[0]['Name'] = row['Name']
    new_row = pd.DataFrame(newlst)
    stats_df = pd.concat([stats_df, new_row])
print(stats_df)             

            Base Stat Total   HP Attack Defense Special Attack  \
0   Base Stats - Total: 318   55     68      64             45   
0   Base Stats - Total: 405   75     89      85             55   
0   Base Stats - Total: 525   95    109     105             75   
0   Base Stats - Total: 309   44     58      44             58   
0   Base Stats - Total: 405   64     78      52             78   
..                      ...  ...    ...     ...            ...   
0   Base Stats - Total: 580   80    105     105            105   
0   Base Stats - Total: 580   75    125      70            125   
0   Base Stats - Total: 680  100    100     120            150   
0   Base Stats - Total: 680   90    100     100            150   
0   Base Stats - Total: 600  100    100     100            100   

   Special Defense Speed      Name  
0               55    31   turtwig  
0               65    36    grotle  
0               85    56  torterra  
0               44    61  chimchar  
0               52    

In [12]:
stats_df.reset_index(drop=True, inplace=True)
print(stats_df)

             Base Stat Total   HP Attack Defense Special Attack  \
0    Base Stats - Total: 318   55     68      64             45   
1    Base Stats - Total: 405   75     89      85             55   
2    Base Stats - Total: 525   95    109     105             75   
3    Base Stats - Total: 309   44     58      44             58   
4    Base Stats - Total: 405   64     78      52             78   
..                       ...  ...    ...     ...            ...   
146  Base Stats - Total: 580   80    105     105            105   
147  Base Stats - Total: 580   75    125      70            125   
148  Base Stats - Total: 680  100    100     120            150   
149  Base Stats - Total: 680   90    100     100            150   
150  Base Stats - Total: 600  100    100     100            100   

    Special Defense Speed      Name  
0                55    31   turtwig  
1                65    36    grotle  
2                85    56  torterra  
3                44    61  chimchar  
4    

In [13]:
new_column = stats_df["Base Stat Total"].str.split(" ", expand = True)
print(new_column)
stats_df['Total'] = new_column[4]
stats_df.drop(columns =["Base Stat Total"], inplace = True)
print(stats_df)

        0      1  2       3    4
0    Base  Stats  -  Total:  318
1    Base  Stats  -  Total:  405
2    Base  Stats  -  Total:  525
3    Base  Stats  -  Total:  309
4    Base  Stats  -  Total:  405
..    ...    ... ..     ...  ...
146  Base  Stats  -  Total:  580
147  Base  Stats  -  Total:  580
148  Base  Stats  -  Total:  680
149  Base  Stats  -  Total:  680
150  Base  Stats  -  Total:  600

[151 rows x 5 columns]
      HP Attack Defense Special Attack Special Defense Speed      Name Total
0     55     68      64             45              55    31   turtwig   318
1     75     89      85             55              65    36    grotle   405
2     95    109     105             75              85    56  torterra   525
3     44     58      44             58              44    61  chimchar   309
4     64     78      52             78              52    81  monferno   405
..   ...    ...     ...            ...             ...   ...       ...   ...
146   80    105     105            105   

In [14]:

cols_to_order = ['Name']
new_columns = cols_to_order + (stats_df.columns.drop(cols_to_order).tolist())
stats_df = stats_df[new_columns]
print(stats_df)


         Name   HP Attack Defense Special Attack Special Defense Speed Total
0     turtwig   55     68      64             45              55    31   318
1      grotle   75     89      85             55              65    36   405
2    torterra   95    109     105             75              85    56   525
3    chimchar   44     58      44             58              44    61   309
4    monferno   64     78      52             78              52    81   405
..        ...  ...    ...     ...            ...             ...   ...   ...
146   mesprit   80    105     105            105             105    80   580
147     azelf   75    125      70            125              70   115   580
148    dialga  100    100     120            150             120    90   680
149    palkia   90    100     100            150             120   120   680
150   manaphy  100    100     100            100             100   100   600

[151 rows x 8 columns]


In [15]:
complete_df = pokemon_df.merge(stats_df, on='Name')

In [16]:
complete_df[['HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed']] = complete_df[['HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed']].apply(pd.to_numeric)


In [17]:
highest_stat = complete_df[['HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed']].idxmax(axis=1)
print(highest_stat)

0              Attack
1              Attack
2              Attack
3               Speed
4               Speed
            ...      
146            Attack
147            Attack
148    Special Attack
149    Special Attack
150                HP
Length: 151, dtype: object


In [18]:
complete_df['Strongest Stat'] = highest_stat
print(complete_df)

    Pokédex no.      Name                      URL              Type   HP  \
0          #001   turtwig   /pokedex-swsh/turtwig/           [Grass]   55   
1          #002    grotle    /pokedex-swsh/grotle/           [Grass]   75   
2          #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]   95   
3          #004  chimchar  /pokedex-swsh/chimchar/            [Fire]   44   
4          #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]   64   
..          ...       ...                      ...               ...  ...   
146        #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]   80   
147        #148     azelf     /pokedex-swsh/azelf/         [Psychic]   75   
148        #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]  100   
149        #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]   90   
150        #151   manaphy   /pokedex-swsh/manaphy/           [Water]  100   

     Attack  Defense  Special Attack  Special Defense  Speed Total  \
0    

In [19]:
resistances = {
    'Normal': 1, 
    'Fire': 1, 
    'Water': 1, 
    'Grass': 1, 
    'Flying': 1, 
    'Fighting': 1, 
    'Poison': 1, 
    'Electric': 1, 
    'Ground': 1, 
    'Rock': 1, 
    'Psychic': 1, 
    'Ice': 1, 
    'Bug': 1, 
    'Ghost': 1, 
    'Steel': 1, 
    'Dragon': 1, 
    'Dark': 1,
    'Fairy': 1
}

In [22]:
primary_types = []
secondary_types = []
for index, row in complete_df.iterrows():
    if (len(row['Type']) == 2):
        primary_types.append(row['Type'][0])
        secondary_types.append(row['Type'][1])
    else:
        primary_types.append(row['Type'][0])
        secondary_types.append('')
        
print(primary_types)
print(secondary_types)

['Grass', 'Grass', 'Grass', 'Fire', 'Fire', 'Fire', 'Water', 'Water', 'Water', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Bug', 'Bug', 'Electric', 'Electric', 'Electric', 'Psychic', 'Psychic', 'Psychic', 'Water', 'Water', 'Grass', 'Grass', 'Grass', 'Poison', 'Poison', 'Poison', 'Rock', 'Rock', 'Rock', 'Rock', 'Steel', 'Rock', 'Rock', 'Rock', 'Rock', 'Fighting', 'Fighting', 'Fighting', 'Water', 'Water', 'Bug', 'Bug', 'Bug', 'Bug', 'Bug', 'Bug', 'Bug', 'Bug', 'Bug', 'Bug', 'Electric', 'Water', 'Water', 'Grass', 'Grass', 'Water', 'Water', 'Bug', 'Normal', 'Normal', 'Ghost', 'Ghost', 'Normal', 'Normal', 'Ghost', 'Ghost', 'Ghost', 'Ghost', 'Ghost', 'Dark', 'Dark', 'Normal', 'Normal', 'Water', 'Water', 'Water', 'Water', 'Psychic', 'Psychic', 'Poison', 'Poison', 'Fighting', 'Fighting', 'Steel', 'Steel', 'Fire', 'Fire', 'Rock', 'Rock', 'Psychic', 'Psychic', 'Normal', 'Normal', 'Normal', 'Fairy', 'Fairy', 'Fairy', 'Normal', 'Electric', 'Electric', 'Electric', 'Normal', 'Normal', 'Ghost'

In [23]:
complete_df['Primary Type'] = primary_types
complete_df['Secondary Type'] = secondary_types

In [26]:
resistance_list = []
for index, row in complete_df.iterrows():
    new_resistances = {
    'Normal': 1, 
    'Fire': 1, 
    'Water': 1, 
    'Grass': 1, 
    'Flying': 1, 
    'Fighting': 1, 
    'Poison': 1, 
    'Electric': 1, 
    'Ground': 1, 
    'Rock': 1, 
    'Psychic': 1, 
    'Ice': 1, 
    'Bug': 1, 
    'Ghost': 1, 
    'Steel': 1, 
    'Dragon': 1, 
    'Dark': 1,
    'Fairy': 1}
    if row['Primary Type'] == 'Normal':
        new_resistances['Ghost'] = 0
        new_resistances['Fighting']*=2
    elif row['Primary Type'] == 'Fire':
        new_resistances['Fire']/=2
        new_resistances['Grass']/=2
        new_resistances['Ice']/=2
        new_resistances['Bug']/=2
        new_resistances['Steel']/=2
        new_resistances['Fairy']/=2
        new_resistances['Water']*=2
        new_resistances['Ground']*=2
        new_resistances['Rock']*=2
    elif row['Primary Type'] == 'Water':
        new_resistances['Fire']/=2
        new_resistances['Water']/=2
        new_resistances['Ice']/=2
        new_resistances['Steel']/=2
        new_resistances['Grass']*=2
        new_resistances['Electric']*=2
    elif row['Primary Type'] == 'Electric':
        new_resistances['Electric']/=2
        new_resistances['Flying']/=2
        new_resistances['Steel']/=2
        new_resistances['Ground']*=2
    elif row['Primary Type'] == 'Grass':
        new_resistances['Water']/=2
        new_resistances['Electric']/=2
        new_resistances['Grass']/=2
        new_resistances['Ground']/=2
        new_resistances['Fire']*=2
        new_resistances['Ice']*=2
        new_resistances['Poison']*=2
        new_resistances['Flying']*=2
        new_resistances['Bug']*=2
    elif row['Primary Type'] == 'Ice':
        new_resistances['Ice']/=2
        new_resistances['Fire']*=2
        new_resistances['Fighting']*=2
        new_resistances['Rock']*=2
        new_resistances['Steel']*=2
    elif row['Primary Type'] == 'Fighting':
        new_resistances['Bug']/=2
        new_resistances['Rock']/=2
        new_resistances['Dark']/=2
        new_resistances['Flying']*=2
        new_resistances['Psychic']*=2
        new_resistances['Fairy']*=2
    elif row['Primary Type'] == 'Poison':
        new_resistances['Grass']/=2
        new_resistances['Fighting']/=2
        new_resistances['Poison']/=2
        new_resistances['Bug']/=2
        new_resistances['Fairy']/=2
        new_resistances['Psychic']*=2
        new_resistances['Ground']*=2
    elif row['Primary Type'] == 'Ground':
        new_resistances['Electric'] = 0
        new_resistances['Poison']/=2
        new_resistances['Rock']/=2
        new_resistances['Water']*=2
        new_resistances['Grass']*=2
        new_resistances['Ice']*=2
    elif row['Primary Type'] == 'Flying':
        new_resistances['Ground'] = 0
        new_resistances['Grass']/=2
        new_resistances['Fighting']/=2
        new_resistances['Bug']/=2
        new_resistances['Electric']*=2
        new_resistances['Ice']*=2
        new_resistances['Rock']*=2
    elif row['Primary Type'] == 'Psychic':
        new_resistances['Fighting']/=2
        new_resistances['Psychic']/=2
        new_resistances['Bug']*=2
        new_resistances['Ghost']*=2
        new_resistances['Dark']*=2
    elif row['Primary Type'] == 'Bug':
        new_resistances['Grass']/=2
        new_resistances['Fighting']/=2
        new_resistances['Ground']/=2
        new_resistances['Fire']*=2
        new_resistances['Flying']*=2
        new_resistances['Rock']*=2
    elif row['Primary Type'] == 'Rock':
        new_resistances['Normal']/=2
        new_resistances['Fire']/=2
        new_resistances['Poison']/=2
        new_resistances['Flying']/=2
        new_resistances['Water']*=2
        new_resistances['Grass']*=2
        new_resistances['Fighting']*=2
        new_resistances['Ground']*=2
        new_resistances['Steel']*=2
    elif row['Primary Type'] == 'Ghost':
        new_resistances['Normal'] = 0
        new_resistances['Fighting'] = 0
        new_resistances['Poison']/=2
        new_resistances['Bug']/=2
        new_resistances['Ghost']*=2
        new_resistances['Dark']*=2
    elif row['Primary Type'] == 'Dragon':
        new_resistances['Fire']/=2
        new_resistances['Water']/=2
        new_resistances['Electric']/=2
        new_resistances['Grass']/=2
        new_resistances['Ice']*=2
        new_resistances['Dragon']*=2
        new_resistances['Fairy']*=2
    elif row['Primary Type'] == 'Dark':
        new_resistances['Psychic'] = 0
        new_resistances['Ghost']/=2
        new_resistances['Dark']/=2
        new_resistances['Fighting']*=2
        new_resistances['Bug']*=2
        new_resistances['Fairy']*=2
    elif row['Primary Type'] == 'Steel':
        new_resistances['Poison'] = 0
        new_resistances['Normal']/=2
        new_resistances['Grass']/=2
        new_resistances['Ice']/=2
        new_resistances['Flying']/=2
        new_resistances['Psychic']/=2
        new_resistances['Bug']/=2
        new_resistances['Rock']/=2
        new_resistances['Dragon']/=2
        new_resistances['Steel']/=2
        new_resistances['Fairy']/=2
        new_resistances['Fire']*=2
        new_resistances['Fighting']*=2
        new_resistances['Ground']*=2
    else:
        new_resistances['Dragon'] = 0
        new_resistances['Fighting']/=2
        new_resistances['Bug']/=2
        new_resistances['Dark']/=2
        new_resistances['Poison']*=2
        new_resistances['Steel']*=2
# For dual type Pokemon only
    if len(row['Type']) == 2:
        if row['Secondary Type'] == 'Normal':
            new_resistances['Ghost'] = 0
            new_resistances['Fighting']*=2
        elif row['Secondary Type'] == 'Fire':
            new_resistances['Fire']/=2
            new_resistances['Grass']/=2
            new_resistances['Ice']/=2
            new_resistances['Bug']/=2
            new_resistances['Steel']/=2
            new_resistances['Fairy']/=2
            new_resistances['Water']*=2
            new_resistances['Ground']*=2
            new_resistances['Rock']*=2
        elif row['Secondary Type'] == 'Water':
            new_resistances['Fire']/=2
            new_resistances['Water']/=2
            new_resistances['Ice']/=2
            new_resistances['Steel']/=2
            new_resistances['Grass']*=2
            new_resistances['Electric']*=2
        elif row['Secondary Type'] == 'Electric':
            new_resistances['Electric']/=2
            new_resistances['Flying']/=2
            new_resistances['Steel']/=2
            new_resistances['Ground']*=2
        elif row['Secondary Type'] == 'Grass':
            new_resistances['Water']/=2
            new_resistances['Electric']/=2
            new_resistances['Grass']/=2
            new_resistances['Ground']/=2
            new_resistances['Fire']*=2
            new_resistances['Ice']*=2
            new_resistances['Poison']*=2
            new_resistances['Flying']*=2
            new_resistances['Bug']*=2
        elif row['Secondary Type'] == 'Ice':
            new_resistances['Ice']/=2
            new_resistances['Fire']*=2
            new_resistances['Fighting']*=2
            new_resistances['Rock']*=2
            new_resistances['Steel']*=2
        elif row['Secondary Type'] == 'Fighting':
            new_resistances['Bug']/=2
            new_resistances['Rock']/=2
            new_resistances['Dark']/=2
            new_resistances['Flying']*=2
            new_resistances['Psychic']*=2
            new_resistances['Fairy']*=2
        elif row['Secondary Type'] == 'Poison':
            new_resistances['Grass']/=2
            new_resistances['Fighting']/=2
            new_resistances['Poison']/=2
            new_resistances['Bug']/=2
            new_resistances['Fairy']/=2
            new_resistances['Psychic']*=2
            new_resistances['Ground']*=2
        elif row['Secondary Type'] == 'Ground':
            new_resistances['Electric'] = 0
            new_resistances['Poison']/=2
            new_resistances['Rock']/=2
            new_resistances['Water']*=2
            new_resistances['Grass']*=2
            new_resistances['Ice']*=2
        elif row['Secondary Type'] == 'Flying':
            new_resistances['Ground'] = 0
            new_resistances['Grass']/=2
            new_resistances['Fighting']/=2
            new_resistances['Bug']/=2
            new_resistances['Electric']*=2
            new_resistances['Ice']*=2
            new_resistances['Rock']*=2
        elif row['Secondary Type'] == 'Psychic':
            new_resistances['Fighting']/=2
            new_resistances['Psychic']/=2
            new_resistances['Bug']*=2
            new_resistances['Ghost']*=2
            new_resistances['Dark']*=2
        elif row['Secondary Type'] == 'Bug':
            new_resistances['Grass']/=2
            new_resistances['Fighting']/=2
            new_resistances['Ground']/=2
            new_resistances['Fire']*=2
            new_resistances['Flying']*=2
            new_resistances['Rock']*=2
        elif row['Secondary Type'] == 'Rock':
            new_resistances['Normal']/=2
            new_resistances['Fire']/=2
            new_resistances['Poison']/=2
            new_resistances['Flying']/=2
            new_resistances['Water']*=2
            new_resistances['Grass']*=2
            new_resistances['Fighting']*=2
            new_resistances['Ground']*=2
            new_resistances['Steel']*=2
        elif row['Secondary Type'] == 'Ghost':
            new_resistances['Normal'] = 0
            new_resistances['Fighting'] = 0
            new_resistances['Poison']/=2
            new_resistances['Bug']/=2
            new_resistances['Ghost']*=2
            new_resistances['Dark']*=2
        elif row['Secondary Type'] == 'Dragon':
            new_resistances['Fire']/=2
            new_resistances['Water']/=2
            new_resistances['Electric']/=2
            new_resistances['Grass']/=2
            new_resistances['Ice']*=2
            new_resistances['Dragon']*=2
            new_resistances['Fairy']*=2
        elif row['Secondary Type'] == 'Dark':
            new_resistances['Psychic'] = 0
            new_resistances['Ghost']/=2
            new_resistances['Dark']/=2
            new_resistances['Fighting']*=2
            new_resistances['Bug']*=2
            new_resistances['Fairy']*=2
        elif row['Secondary Type'] == 'Steel':
            new_resistances['Poison'] = 0
            new_resistances['Normal']/=2
            new_resistances['Grass']/=2
            new_resistances['Ice']/=2
            new_resistances['Flying']/=2
            new_resistances['Psychic']/=2
            new_resistances['Bug']/=2
            new_resistances['Rock']/=2
            new_resistances['Dragon']/=2
            new_resistances['Steel']/=2
            new_resistances['Fairy']/=2
            new_resistances['Fire']*=2
            new_resistances['Fighting']*=2
            new_resistances['Ground']*=2
        else:
            new_resistances['Dragon'] = 0
            new_resistances['Fighting']/=2
            new_resistances['Bug']/=2
            new_resistances['Dark']/=2
            new_resistances['Poison']*=2
            new_resistances['Steel']*=2
    resistance_list.append(new_resistances)
        

In [27]:
print(resistance_list)

[{'Normal': 1, 'Fire': 2, 'Water': 0.5, 'Grass': 0.5, 'Flying': 2, 'Fighting': 1, 'Poison': 2, 'Electric': 0.5, 'Ground': 0.5, 'Rock': 1, 'Psychic': 1, 'Ice': 2, 'Bug': 2, 'Ghost': 1, 'Steel': 1, 'Dragon': 1, 'Dark': 1, 'Fairy': 1}, {'Normal': 1, 'Fire': 2, 'Water': 0.5, 'Grass': 0.5, 'Flying': 2, 'Fighting': 1, 'Poison': 2, 'Electric': 0.5, 'Ground': 0.5, 'Rock': 1, 'Psychic': 1, 'Ice': 2, 'Bug': 2, 'Ghost': 1, 'Steel': 1, 'Dragon': 1, 'Dark': 1, 'Fairy': 1}, {'Normal': 1, 'Fire': 2, 'Water': 1.0, 'Grass': 1.0, 'Flying': 2, 'Fighting': 1, 'Poison': 1.0, 'Electric': 0, 'Ground': 0.5, 'Rock': 0.5, 'Psychic': 1, 'Ice': 4, 'Bug': 2, 'Ghost': 1, 'Steel': 1, 'Dragon': 1, 'Dark': 1, 'Fairy': 1}, {'Normal': 1, 'Fire': 0.5, 'Water': 2, 'Grass': 0.5, 'Flying': 1, 'Fighting': 1, 'Poison': 1, 'Electric': 1, 'Ground': 2, 'Rock': 2, 'Psychic': 1, 'Ice': 0.5, 'Bug': 0.5, 'Ghost': 1, 'Steel': 0.5, 'Dragon': 1, 'Dark': 1, 'Fairy': 0.5}, {'Normal': 1, 'Fire': 0.5, 'Water': 2, 'Grass': 0.5, 'Flying': 2,

In [28]:
complete_df['Resistances'] = resistance_list
print(complete_df)

    Pokédex no.      Name                      URL              Type   HP  \
0          #001   turtwig   /pokedex-swsh/turtwig/           [Grass]   55   
1          #002    grotle    /pokedex-swsh/grotle/           [Grass]   75   
2          #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]   95   
3          #004  chimchar  /pokedex-swsh/chimchar/            [Fire]   44   
4          #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]   64   
..          ...       ...                      ...               ...  ...   
146        #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]   80   
147        #148     azelf     /pokedex-swsh/azelf/         [Psychic]   75   
148        #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]  100   
149        #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]   90   
150        #151   manaphy   /pokedex-swsh/manaphy/           [Water]  100   

     Attack  Defense  Special Attack  Special Defense  Speed Total  \
0    

In [41]:
abilities = []
hidden_abilities = []
for index, row in complete_df.iterrows():
    if row['Name'] == 'wormadam':
        url = "https://pokeapi.co/api/v2/pokemon/413/"
    elif row['Name'] == 'mimejr.':
        url = "https://pokeapi.co/api/v2/pokemon/mime-jr/"
    elif row['Name'] == 'mr.mime':
        url = "https://pokeapi.co/api/v2/pokemon/mr-mime/"
    else:
        url = "https://pokeapi.co/api/v2/pokemon/" + row['Name']
    response = requests.get(url)
    #print(row['Name'])
    ability_map = response.json()['abilities']
    primary_ability = []
    hidden_ability = 'None'
    for ability in ability_map:
        #print(ability)
        if ability['is_hidden']:
            hidden_ability = ability['ability']['name'].replace('-', ' ')
        else:
            primary_ability.append(ability['ability']['name'].replace('-', ' '))
    abilities.append(primary_ability)
    hidden_abilities.append(hidden_ability)


In [42]:
complete_df['Ability'] = abilities
complete_df['Hidden Ability'] = hidden_abilities

In [43]:
print(complete_df)

    Pokédex no.      Name                      URL              Type   HP  \
0          #001   turtwig   /pokedex-swsh/turtwig/           [Grass]   55   
1          #002    grotle    /pokedex-swsh/grotle/           [Grass]   75   
2          #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]   95   
3          #004  chimchar  /pokedex-swsh/chimchar/            [Fire]   44   
4          #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]   64   
..          ...       ...                      ...               ...  ...   
146        #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]   80   
147        #148     azelf     /pokedex-swsh/azelf/         [Psychic]   75   
148        #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]  100   
149        #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]   90   
150        #151   manaphy   /pokedex-swsh/manaphy/           [Water]  100   

     Attack  Defense  Special Attack  Special Defense  Speed Total  \
0    

In [49]:
resistances = {
    'Normal': [], 
    'Fire': [], 
    'Water': [], 
    'Grass': [], 
    'Flying': [], 
    'Fighting': [], 
    'Poison': [], 
    'Electric': [], 
    'Ground': [], 
    'Rock': [], 
    'Psychic': [], 
    'Ice': [], 
    'Bug': [], 
    'Ghost': [], 
    'Steel': [], 
    'Dragon': [], 
    'Dark': [],
    'Fairy': []
}
for index, row in complete_df.iterrows():
    resistances['Normal'].append(row['Resistances']['Normal'])
    resistances['Fire'].append(row['Resistances']['Fire'])
    resistances['Water'].append(row['Resistances']['Water'])
    resistances['Grass'].append(row['Resistances']['Grass'])
    resistances['Flying'].append(row['Resistances']['Flying'])
    resistances['Fighting'].append(row['Resistances']['Fighting'])
    resistances['Poison'].append(row['Resistances']['Poison'])
    resistances['Electric'].append(row['Resistances']['Electric'])
    resistances['Ground'].append(row['Resistances']['Ground'])
    resistances['Rock'].append(row['Resistances']['Rock'])
    resistances['Psychic'].append(row['Resistances']['Psychic'])
    resistances['Ice'].append(row['Resistances']['Ice'])
    resistances['Bug'].append(row['Resistances']['Bug'])
    resistances['Ghost'].append(row['Resistances']['Ghost'])
    resistances['Steel'].append(row['Resistances']['Steel'])
    resistances['Dragon'].append(row['Resistances']['Dragon'])
    resistances['Dark'].append(row['Resistances']['Dark'])
    resistances['Fairy'].append(row['Resistances']['Fairy'])



In [50]:
complete_df['Normal'] = resistances['Normal']
complete_df['Fire'] = resistances['Fire']
complete_df['Water'] = resistances['Water']
complete_df['Grass'] = resistances['Grass']
complete_df['Flying'] = resistances['Flying']
complete_df['Fighting'] = resistances['Fighting']
complete_df['Poison'] = resistances['Poison']
complete_df['Electric'] = resistances['Electric']
complete_df['Ground'] = resistances['Ground']
complete_df['Rock'] = resistances['Rock']
complete_df['Psychic'] = resistances['Psychic']
complete_df['Ice'] = resistances['Ice']
complete_df['Bug'] = resistances['Bug']
complete_df['Ghost'] = resistances['Ghost']
complete_df['Steel'] = resistances['Steel']
complete_df['Dragon'] = resistances['Dragon']
complete_df['Dark'] = resistances['Dark']
complete_df['Fairy'] = resistances['Fairy']

In [51]:
print(complete_df)

    Pokédex no.      Name                      URL              Type   HP  \
0          #001   turtwig   /pokedex-swsh/turtwig/           [Grass]   55   
1          #002    grotle    /pokedex-swsh/grotle/           [Grass]   75   
2          #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]   95   
3          #004  chimchar  /pokedex-swsh/chimchar/            [Fire]   44   
4          #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]   64   
..          ...       ...                      ...               ...  ...   
146        #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]   80   
147        #148     azelf     /pokedex-swsh/azelf/         [Psychic]   75   
148        #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]  100   
149        #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]   90   
150        #151   manaphy   /pokedex-swsh/manaphy/           [Water]  100   

     Attack  Defense  Special Attack  Special Defense  Speed  ... Ground Ro

In [52]:
complete_df.to_json(r'./pokemon.json')

In [62]:
imageUrls = []
for index, row in complete_df.iterrows():
    if row['Name'] == 'wormadam':
        url = "https://pokeapi.co/api/v2/pokemon/413/"
    elif row['Name'] == 'mimejr.':
        url = "https://pokeapi.co/api/v2/pokemon/mime-jr/"
    elif row['Name'] == 'mr.mime':
        url = "https://pokeapi.co/api/v2/pokemon/mr-mime/"
    else:
        url = "https://pokeapi.co/api/v2/pokemon/" + row['Name']
    response = requests.get(url)
    print(row['Name'])
    imageUrl = response.json()['sprites']['front_default']
    print(imageUrl)
    print()
    imageUrls.append(imageUrl)

turtwig
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/387.png

grotle
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/388.png

torterra
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/389.png

chimchar
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/390.png

monferno
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/391.png

infernape
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/392.png

piplup
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/393.png

prinplup
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/394.png

empoleon
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/395.png

starly
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/396.png

staravia
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/397.png

stara

mimejr.
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/439.png

mr.mime
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/122.png

happiny
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/440.png

chansey
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/113.png

blissey
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/242.png

cleffa
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/173.png

clefairy
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/35.png

clefable
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/36.png

chatot
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/441.png

pichu
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/172.png

pikachu
https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/25.png

raichu
https://

In [63]:
complete_df['Sprite URL'] = imageUrls
print(complete_df)

    Pokédex no.      Name                      URL              Type   HP  \
0          #001   turtwig   /pokedex-swsh/turtwig/           [Grass]   55   
1          #002    grotle    /pokedex-swsh/grotle/           [Grass]   75   
2          #003  torterra  /pokedex-swsh/torterra/   [Grass, Ground]   95   
3          #004  chimchar  /pokedex-swsh/chimchar/            [Fire]   44   
4          #005  monferno  /pokedex-swsh/monferno/  [Fire, Fighting]   64   
..          ...       ...                      ...               ...  ...   
146        #147   mesprit   /pokedex-swsh/mesprit/         [Psychic]   80   
147        #148     azelf     /pokedex-swsh/azelf/         [Psychic]   75   
148        #149    dialga    /pokedex-swsh/dialga/   [Steel, Dragon]  100   
149        #150    palkia    /pokedex-swsh/palkia/   [Water, Dragon]   90   
150        #151   manaphy   /pokedex-swsh/manaphy/           [Water]  100   

     Attack  Defense  Special Attack  Special Defense  Speed Total  \
0    

In [67]:
for index, row in complete_df.iterrows():
    image_url = row['Sprite URL']
    img_data = requests.get(image_url).content
    filename = './sprites/' + row['Name'] + '.png'
    with open(filename, 'wb') as handler:
        handler.write(img_data)